In [2]:
text = open("input.txt", 'r').read()
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [7]:
vocab = sorted(list(set(''.join(text))))
print(''.join(vocab))

stoi = {s:i for i,s in enumerate(vocab)}
itos = {i:s for i,s in enumerate(vocab)}

encode = lambda s: [stoi[ch] for ch in s]
print(encode('hi there!'))

decode = lambda l: ''.join([itos[i] for i in l])
print(decode(encode('hi there!')))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
[46, 47, 1, 58, 46, 43, 56, 43, 2]
hi there!
